### Tutorial: How to Use the Given Code

This notebook is a guide to using the code provided in our package. The first thing we would need to do is to download the package onto your computer. After this, you can simply import the package using the command:

In [9]:
%load_ext autoreload
%autoreload 2
import os, sys
import pandas as pd
import MCAT_pkg as mc 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


After we import our package, we will need to load in our data. 

In [16]:
# Identify the location that the data file is in  
data_path = "data/gardner_mt_catastrophe_only_tubulin.csv"

# Read in the file as df
df = pd.read_csv(data_path, comment = "#")

In [17]:
MCAT_pkg?

Type:        module
String form: <module 'MCAT_pkg' from '/Users/meiyiyou/Desktop/GItHub/bebi103/MCAT/MCAT_pkg/__init__.py'>
File:        ~/Desktop/GItHub/bebi103/MCAT/MCAT_pkg/__init__.py
Docstring:   <no docstring>
